In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
import time
from matplotlib import rc

In [ ]:
# Set font size to 14
plt.rcParams.update({'font.size': 16})

# Set font to Times New Roman
rc('font', family='Times New Roman')

# Read the dataset
data = pd.read_excel('RFE递归含mateminer.xlsx')

# Extract features and target variable
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=3)

# Initialize XGBoost regressor
xgb = XGBRegressor()

# Record the initial dimension before feature elimination
initial_dim = X_train.shape[1]

# Define empty lists to store MAE, RMSE, and time for each dimension
mae_scores = []
rmse_scores = []
time_taken = []

# Recursive feature elimination
for i in range(initial_dim, 3, -1):
    # Create RFE object with XGBoost regressor as the base model, removing 1 feature at a time
    rfe = RFE(estimator=xgb, n_features_to_select=i, step=1)
    
    # Fit RFE object to the training set
    start_time = time.time()
    rfe.fit(X_train, y_train)
    end_time = time.time()
    
    # Extract the selected features from RFE
    selected_features = X_train.columns[rfe.support_]
    
    # Create new training and testing sets with the selected features
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]
    
    # Train the XGBoost regressor
    start_model_time = time.time()
    xgb.fit(X_train_selected, y_train)
    end_model_time = time.time()
    
    # Make predictions on the test set
    y_pred = xgb.predict(X_test_selected)
    
    # Calculate MAE, RMSE, and model training time
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    model_time_taken = end_model_time - start_model_time
    
    # Append MAE, RMSE, and model training time to the lists
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    time_taken.append(end_time - start_time)

# Reverse the lists to plot X-axis from left to right
mae_scores.reverse()
rmse_scores.reverse()
time_taken.reverse()

# Define colors for each feature type
colors = ['#CD5C5C', '#5D98A6', '#F2C36B']

# Plot the line chart
fig, ax1 = plt.subplots(figsize=(8,6))
ax2 = ax1.twinx()

for i, (score, color) in enumerate(zip([mae_scores, rmse_scores], colors)):
    ax1.plot(range(initial_dim, 3, -1), score, marker='o', linestyle='-', color=color, label=['MAE', 'RMSE'][i])
    ax1.set_xlabel('Number of Dimensions')
    ax1.set_ylabel('Error', color='black')
    ax1.tick_params('y', colors='black')

ax2.plot(range(initial_dim, 3, -1), time_taken, marker='o', linestyle='-', color='green', label='Time Taken')
ax2.set_ylabel('Time Taken (seconds)', color='green')
ax2.tick_params('y', colors='green')

# Move RMSE and MAE labels to the right upper corner
handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(handles1 + handles2, labels1 + labels2, loc='upper right')

plt.title('MAE, RMSE, and Time Taken for Dimension Reduction')
plt.gca().invert_xaxis()  # Invert X-axis
plt.show()
